In [2]:
pip install faiss-cpu sentence-transformers cohere


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\program files\python312\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [1]:
from sentence_transformers import SentenceTransformer

# Example document corpus
documents = [
    "Pinecone is a vector database for machine learning applications.",
    "Cohere provides APIs for natural language processing tasks like text generation.",
    "Sentence Transformers is a library that allows easy embedding of sentences."
]

# Load a pre-trained sentence transformer model to generate embeddings
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

# Generate embeddings for the documents
embeddings = model.encode(documents)


C:\Users\deepa\AppData\Roaming\Python\Python312\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


C:\Users\deepa\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\deepa\.cache\huggingface\hub\models--sentence-transformers--multi-qa-MiniLM-L6-cos-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
C:\Users\deepa\AppData\Roaming\Python\Python312\site-packages\transformers\to

In [2]:
import faiss
import numpy as np

# Get the dimensionality of embeddings
embedding_dim = embeddings.shape[1]

# Initialize a FAISS index (Flat Index is simplest for small datasets)
index = faiss.IndexFlatL2(embedding_dim)

# Convert embeddings to numpy array (required by FAISS)
embeddings = np.array(embeddings)

# Add document embeddings to the FAISS index
index.add(embeddings)

# Check the total number of vectors in the index
print(f"Total documents in index: {index.ntotal}")


Total documents in index: 3


In [3]:
# Example query from the user
query = "What is Pinecone?"

# Convert the query into an embedding
query_embedding = model.encode([query])

# Perform similarity search in FAISS (returns indices of top k results)
k = 3  # Number of nearest neighbors to retrieve
distances, indices = index.search(np.array(query_embedding), k)

# Print out the retrieved documents
retrieved_docs = [documents[i] for i in indices[0]]
print("Retrieved Documents:", retrieved_docs)


Retrieved Documents: ['Pinecone is a vector database for machine learning applications.', 'Cohere provides APIs for natural language processing tasks like text generation.', 'Sentence Transformers is a library that allows easy embedding of sentences.']


In [6]:
import cohere

# Initialize Cohere with your API key
cohere_client = cohere.Client('qFR72UQsWpnuLJW4cU7bP6R10X4sBd4lZNszGTJN')

# Combine retrieved documents into a single context
retrieved_context = " ".join(retrieved_docs)

# Use Cohere's generation API to create a coherent answer
response = cohere_client.generate(
    model='command-xlarge-nightly',
    prompt=f"Question: {query}\nContext: {retrieved_context}\nAnswer:",
    max_tokens=100
)

# Print the generated answer
print("Generated Answer:", response.generations[0].text.strip())


Generated Answer: Pinecone is a vector database for machine learning applications.


In [8]:
queries = ["Tell me about Cohere.", "What can Sentence Transformers do?", "What is a vector database?"]

for q in queries:
    # Convert each query into an embedding
    query_embedding = model.encode([q])
    
    # Retrieve relevant documents from FAISS
    distances, indices = index.search(np.array(query_embedding), k)
    retrieved_docs = [documents[i] for i in indices[0]]
    
    # Create the context from retrieved documents
    retrieved_context = " ".join(retrieved_docs)
    
    # Generate the answer using Cohere
    response = cohere_client.generate(
        model='command-xlarge-nightly',
        prompt=f"Question: {q}\nContext: {retrieved_context}\nAnswer:",
        max_tokens=100
    )
    
    print(f"Question: {q}")
    print(f"Generated Answer: {response.generations[0].text.strip()}")
    print("-" * 50)


Question: Tell me about Cohere.
Generated Answer: Cohere is a Canadian startup that provides natural language processing (NLP) models for text generation, classification, and search. It was founded in 2019 by Aidan Gomez (CEO), Ivan Zhang (CTO), and Nick Frosst to address the growing demand for large language models in various industries.

The company's mission is to make advanced NLP technology accessible to developers and businesses, enabling them to build intelligent applications that can understand and process human language. Cohere's products are designed
--------------------------------------------------
Question: What can Sentence Transformers do?
Generated Answer: Sentence Transformers is a library that allows easy embedding of sentences.
--------------------------------------------------
Question: What is a vector database?
Generated Answer: A vector database is a database that stores vectors and allows for efficient similarity search.
-----------------------------------------

In [9]:
import fitz  # PyMuPDF for handling PDFs
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import cohere

# Load Cohere API and Sentence Transformers
cohere_client = cohere.Client('qFR72UQsWpnuLJW4cU7bP6R10X4sBd4lZNszGTJN')
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

# Function to extract text from PDF
def extract_text_from_pdf(pdf_file):
    doc = fitz.open(pdf_file)
    text = ""
    for page in doc:
        text += page.get_text("text")
    return text

# Function to create embeddings and index with FAISS
def create_faiss_index(documents):
    embeddings = model.encode(documents)
    embedding_dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(embedding_dim)
    embeddings = np.array(embeddings)
    index.add(embeddings)
    return index, embeddings

# Function to search FAISS index
def search_faiss_index(query, index, documents, k=3):
    query_embedding = model.encode([query])
    distances, indices = index.search(np.array(query_embedding), k)
    retrieved_docs = [documents[i] for i in indices[0]]
    return retrieved_docs


C:\Users\deepa\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
